In [1]:
import nltk
import numpy as np
from nltk.corpus import genesis
from nltk.corpus import wordnet as wn
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('genesis')

nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('averaged_perceptron_tagger')
nltk.download('vader_lexicon')

In [2]:
import pandas as pd

path = "./kurrek.2020.slur-corpus.csv"
data = pd.read_csv(path, sep=",", header = 0, on_bad_lines='skip')

In [15]:
# Make function to check if string contains any element from a string
# https://bobbyhadz.com/blog/python-check-if-string-contains-element-from-list
def is_element_in_string(string, flist):
    if any((match := substring) in string for substring in flist):
        print('The string contains at least one element from the list')
    return match

In [44]:
# Replace slur with [REDACTED], and print
# Make new data list using pandas
# This one is only to print the strings, to avoid printing harmful language. 

path = "./kurrek.2020.slur-corpus.csv"
redacted_data = pd.read_csv(path, sep=",", header = 0, on_bad_lines='skip')

# Iterate for each index in redacted_data, and modify the 'body' to replace the 
# slur with [REDACTED]. The indexes will remain the same as the one found in the original
# document, so this is really to just print out certain rows whilst avoiding printing out the
# harmful language (the slurs).
for index, row in redacted_data.iterrows():
    T1, T2 = str(row['body']), str(row['slur'])
    redacted_data.at[index, 'body'] = T1.lower().replace(T2, "[REDACTED]")
    # print a couple to show it works:
    if index % 5000 == 0:
        print(f"Index: {index}\nBody: {redacted_data.at[index, 'body']}\n")


Index: 0
Body: fuck that i dont wanna watch [REDACTED] porn. 

Index: 5000
Body: if i took them to the forums chances are sergei would call me a [REDACTED] or one of the mods would ban me for speaking ill of the game, reddit i feel is the only place i feel i can critique the game safely.

Index: 10000
Body: good idea.  let's just all talk online about it, and ignore it in the real world.  things will definitely change, then.

eat a dick you [REDACTED] ass bitch.

Index: 15000
Body: ayyy boo-boo, it's time to lynch us a [REDACTED]

Index: 20000
Body: yeah, the [REDACTED] work song has really made a comeback in the last few years.

Index: 25000
Body: nah, add in "faggot" "cuck" and "sand [REDACTED]" then you have a t_d spam post. 

Index: 30000
Body: yea, cause it would have taken him up to that point to realize you just spelled out [REDACTED]. 

Index: 35000
Body: good on /u/randsfoodstamps for calling them out about "violent" people. seriously, /r/libertarian has a huge crossover with 

## Task 1

In [ ]:
"""By constructing a dataframe of posts assigned to the same category, suggest a script that outputs the
vocabulary set of each category, the size of the vocabulary, the total number of tokens, the average
number of tokens per post and its standard deviation, the average number of pronouns per post and the
associated standard deviation, the ten most frequent tokens in each category, excluding the stopword list.
Represent the statistical result in a clear table and discuss whether some parameters are most relevant to
discriminate a given category."""

In [1]:
# The following is a list of categories, and the number of frequencies
"""
DEG 	Derogatory 	20531
NDG 	Non Derogatory Non Appropriative 	16729
HOM 	Homonym 	1998
APR 	Appropriative 	553
CMP 	Noise 	189
"""
# Create dictionary variables to store data stored about each category